In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

df= pd.read_csv('C://Users//User//Desktop//MSc Westminster//Dissertation//DataSets//Heart_Disease_Indicators.csv')

In [2]:
from sklearn.model_selection import train_test_split

target_size = 50000

num_class_0 = int(target_size * df['HeartDiseaseorAttack'].value_counts(normalize=True)[0])
num_class_1 = target_size - num_class_0

df_class_0 = df[df['HeartDiseaseorAttack'] == 0]
df_class_1 = df[df['HeartDiseaseorAttack'] == 1]

df_class_0_sampled = df_class_0.sample(n=num_class_0, random_state=15)
df_class_1_sampled = df_class_1.sample(n=num_class_1, random_state=15)

df_sampled = pd.concat([df_class_0_sampled, df_class_1_sampled])

df_sampled = df_sampled.sample(frac=1, random_state=15).reset_index(drop=True)
df= df_sampled
print(df['HeartDiseaseorAttack'].value_counts(normalize=True))

0    0.90582
1    0.09418
Name: HeartDiseaseorAttack, dtype: float64


In [3]:
df.drop_duplicates(inplace= True)
df.shape

(48050, 22)

In [4]:
df= df[df['Diabetes'] != 1].copy()
df.loc[df['Diabetes'] == 2, 'Diabetes'] = 1
print(df['Diabetes'].value_counts())

0    40175
1     6968
Name: Diabetes, dtype: int64


In [5]:
categorical_columns= ['HighBP', 'HighChol', 'CholCheck', 'Smoker', 'Stroke', 'Diabetes', 'PhysActivity',
                      'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'DiffWalk',
                      'Sex', 'Age', 'Education', 'Income'
                     ]
df[categorical_columns]= df[categorical_columns].astype(str)
df= pd.get_dummies(df, columns= categorical_columns, drop_first= True)

In [6]:
X= df.drop(['HeartDiseaseorAttack'], axis= 1)
y= df['HeartDiseaseorAttack']

In [7]:
from collections import Counter
from imblearn.under_sampling import ClusterCentroids
from sklearn.cluster import KMeans

cc= ClusterCentroids(random_state= 15, estimator= KMeans(n_init= 10))

X_cc, y_cc= cc.fit_resample(X, y)
print(f'Cluster Centroids: {Counter(y_cc)}')

Cluster Centroids: Counter({0: 4561, 1: 4561})


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state= 15, stratify= y)

continuous_columns = ['BMI', 'GenHlth', 'MentHlth', 'PhysHlth']

from sklearn.preprocessing import StandardScaler
ss= StandardScaler()
X_train= ss.fit_transform(X_train)
X_test= ss.transform(X_test)

X_cc_train, y_cc_train= cc.fit_resample(X_train, y_train)

--- LogisticRegression ---

In [9]:
from sklearn.metrics import confusion_matrix, classification_report, precision_score, roc_auc_score, accuracy_score
from sklearn.linear_model import LogisticRegression
lr_cc= LogisticRegression(random_state=15)
lr_cc.fit(X_cc_train, y_cc_train)
y_pred_lr_cc= lr_cc.predict(X_test)
y_pred_prob_lr_cc= lr_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_lr_cc, labels= lr_cc.classes_)
print(classification_report(y_test, y_pred_lr_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lr_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lr_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lr_cc))

              precision    recall  f1-score   support

           0       0.97      0.71      0.82      8517
           1       0.23      0.81      0.36       912

    accuracy                           0.72      9429
   macro avg       0.60      0.76      0.59      9429
weighted avg       0.90      0.72      0.78      9429

Precision: 0.23249211356466876
Accuracy: 0.7234065118252201
AUC: 0.8319380331184896


--- DecisionTreeClassifier ---

In [10]:
from sklearn.tree import DecisionTreeClassifier
dt_cc= DecisionTreeClassifier(random_state=15)
dt_cc.fit(X_cc_train, y_cc_train)
y_pred_dt_cc= dt_cc.predict(X_test)
y_pred_prob_dt_cc= dt_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_dt_cc, labels= dt_cc.classes_)
print(classification_report(y_test, y_pred_dt_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_dt_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_dt_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_dt_cc))

              precision    recall  f1-score   support

           0       0.96      0.15      0.26      8517
           1       0.11      0.94      0.19       912

    accuracy                           0.23      9429
   macro avg       0.53      0.55      0.23      9429
weighted avg       0.88      0.23      0.25      9429

Precision: 0.10629629629629629
Accuracy: 0.2268533248488705
AUC: 0.5470658914369404


--- Tuned - DecisionTreeClassifier ---

In [11]:
from sklearn.model_selection import GridSearchCV
param_grid= {
             'max_depth': [None, 10, 20, 30, 40, 50],
             'min_samples_split': [2, 10, 20],
             'min_samples_leaf': [1, 5, 10],
             'max_features': [None, 'sqrt', 'log2'],
             'criterion': ['gini', 'entropy']
            }

gs_dt_cc= GridSearchCV(estimator= dt_cc, param_grid= param_grid, cv= 5, scoring= 'precision')
gs_dt_cc.fit(X_cc_train, y_cc_train)

print("Best Parameters:", gs_dt_cc.best_params_)
print("Best Precision Score:", gs_dt_cc.best_score_)

Best Parameters: {'criterion': 'gini', 'max_depth': None, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 20}
Best Precision Score: 0.9385417960511923


In [12]:
tuned_dt_cc= gs_dt_cc.best_estimator_
y_pred_tuned_dt_cc= tuned_dt_cc.predict(X_test)
y_pred_prob_tuned_dt_cc= tuned_dt_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_dt_cc, labels= tuned_dt_cc.classes_)
print(classification_report(y_test, y_pred_tuned_dt_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_dt_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_dt_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_dt_cc))

              precision    recall  f1-score   support

           0       0.96      0.17      0.28      8517
           1       0.11      0.94      0.19       912

    accuracy                           0.24      9429
   macro avg       0.54      0.55      0.24      9429
weighted avg       0.88      0.24      0.27      9429

Precision: 0.10783329148882752
Accuracy: 0.24064057694347227
AUC: 0.5835586953028928


--- KNeighborsClassifier ---

In [13]:
from sklearn.neighbors import KNeighborsClassifier
knn_cc= KNeighborsClassifier()
knn_cc.fit(X_cc_train, y_cc_train)
y_pred_knn_cc= knn_cc.predict(X_test)
y_pred_prob_knn_cc= knn_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_knn_cc, labels= knn_cc.classes_)
print(classification_report(y_test, y_pred_knn_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_knn_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_knn_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_knn_cc))

              precision    recall  f1-score   support

           0       0.93      0.84      0.88      8517
           1       0.22      0.44      0.30       912

    accuracy                           0.80      9429
   macro avg       0.58      0.64      0.59      9429
weighted avg       0.86      0.80      0.83      9429

Precision: 0.22462562396006655
Accuracy: 0.7979637289214127
AUC: 0.7317146537677998


--- Tuned - KNeighborsClassifier ---

In [14]:
param_grid= {
             'n_neighbors': np.arange(1,40),
             'weights': ['uniform', 'distance'],
             'metric': ['euclidean', 'manhattan', 'minkowski']
            }

gs_knn_cc= GridSearchCV(estimator= knn_cc, param_grid= param_grid, cv=5, scoring= 'precision')
gs_knn_cc.fit(X_cc_train, y_cc_train)
print("Best Parameters:", gs_knn_cc.best_params_)
print("Best Precision Score:", gs_knn_cc.best_score_)

Best Parameters: {'metric': 'manhattan', 'n_neighbors': 8, 'weights': 'uniform'}
Best Precision Score: 0.8423993676265814


In [15]:
tuned_knn_cc= gs_knn_cc.best_estimator_
y_pred_tuned_knn_cc= tuned_knn_cc.predict(X_test)
y_pred_prob_tuned_knn_cc= tuned_knn_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_knn_cc, labels= tuned_knn_cc.classes_)
print(classification_report(y_test, y_pred_tuned_knn_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_knn_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_knn_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_knn_cc))

              precision    recall  f1-score   support

           0       0.97      0.60      0.74      8517
           1       0.18      0.85      0.30       912

    accuracy                           0.62      9429
   macro avg       0.58      0.72      0.52      9429
weighted avg       0.90      0.62      0.70      9429

Precision: 0.1841917873249466
Accuracy: 0.6210626789691378
AUC: 0.7877338717817203


--- GaussianNB ---

In [16]:
from sklearn.naive_bayes import GaussianNB
nb_cc= GaussianNB()
nb_cc.fit(X_cc_train, y_cc_train)
y_pred_nb_cc= nb_cc.predict(X_test)
y_pred_prob_nb_cc= nb_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_nb_cc, labels= nb_cc.classes_)
print(classification_report(y_test, y_pred_nb_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_nb_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_nb_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_nb_cc))

              precision    recall  f1-score   support

           0       0.98      0.39      0.56      8517
           1       0.14      0.91      0.24       912

    accuracy                           0.44      9429
   macro avg       0.56      0.65      0.40      9429
weighted avg       0.90      0.44      0.53      9429

Precision: 0.13863031914893617
Accuracy: 0.44214656909534417
AUC: 0.795080504625424


--- SVM ---

In [17]:
from sklearn.svm import SVC
svc_cc= SVC(kernel= 'rbf',probability= True, gamma= 1, random_state=15)
svc_cc.fit(X_cc_train, y_cc_train)
y_pred_svc_cc= svc_cc.predict(X_test)
y_pred_prob_svc_cc= svc_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_svc_cc, labels= svc_cc.classes_)
print(classification_report(y_test, y_pred_svc_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_svc_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_svc_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_svc_cc))

              precision    recall  f1-score   support

           0       0.91      0.95      0.93      8517
           1       0.13      0.07      0.10       912

    accuracy                           0.86      9429
   macro avg       0.52      0.51      0.51      9429
weighted avg       0.83      0.86      0.85      9429

Precision: 0.13152804642166344
Accuracy: 0.862869869551384
AUC: 0.6058834343696509


--- Random Forest ---

In [18]:
from sklearn.ensemble import RandomForestClassifier
rf_cc= RandomForestClassifier(random_state=15)
rf_cc.fit(X_cc_train, y_cc_train)
y_pred_rf_cc= rf_cc.predict(X_test)
y_pred_prob_rf_cc= rf_cc.predict_proba(X_test)[:,1]

In [19]:
cm= confusion_matrix(y_test, y_pred_rf_cc, labels= rf_cc.classes_)
print(classification_report(y_test, y_pred_rf_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_rf_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_rf_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_rf_cc))

              precision    recall  f1-score   support

           0       1.00      0.06      0.11      8517
           1       0.10      1.00      0.18       912

    accuracy                           0.15      9429
   macro avg       0.55      0.53      0.15      9429
weighted avg       0.91      0.15      0.12      9429

Precision: 0.10183527305282006
Accuracy: 0.14858415526566973
AUC: 0.6496091923480181


In [20]:
param_grid= {
             'n_estimators': [100, 200, 300],
             'max_depth': [None, 10, 20, 30],
             'min_samples_split': [2, 5, 10],
             'min_samples_leaf': [1, 2, 4],
            }

gs_tuned_rf_cc= GridSearchCV(estimator= rf_cc, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_tuned_rf_cc.fit(X_cc_train, y_cc_train)
print("Best Parameters:", gs_tuned_rf_cc.best_params_)
print("Best Precision Score:", gs_tuned_rf_cc.best_score_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 300}
Best Precision Score: 0.94362189277958


In [21]:
tuned_rf_cc= gs_tuned_rf_cc.best_estimator_
y_pred_tuned_rf_cc= tuned_rf_cc.predict(X_test)
y_pred_prob_tuned_rf_cc= tuned_rf_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_rf_cc, labels= tuned_rf_cc.classes_)
print(classification_report(y_test, y_pred_tuned_rf_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_rf_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_rf_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_rf_cc))

              precision    recall  f1-score   support

           0       1.00      0.06      0.11      8517
           1       0.10      1.00      0.18       912

    accuracy                           0.15      9429
   macro avg       0.55      0.53      0.15      9429
weighted avg       0.91      0.15      0.12      9429

Precision: 0.10191296565611366
Accuracy: 0.14847809948032664
AUC: 0.6538729815909975


--- AdaBoost ---

In [22]:
from sklearn.ensemble import AdaBoostClassifier
ada_cc= AdaBoostClassifier(random_state=15)
ada_cc.fit(X_cc_train, y_cc_train)
y_pred_ada_cc= ada_cc.predict(X_test)
y_pred_prob_ada_cc= ada_cc.predict_proba(X_test)[:,1]

In [23]:
cm= confusion_matrix(y_test, y_pred_ada_cc, labels= ada_cc.classes_)
print(classification_report(y_test, y_pred_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_ada_cc))

              precision    recall  f1-score   support

           0       0.94      0.03      0.07      8517
           1       0.10      0.98      0.18       912

    accuracy                           0.13      9429
   macro avg       0.52      0.51      0.12      9429
weighted avg       0.86      0.13      0.08      9429

Precision: 0.09788446782856516
Accuracy: 0.12514582670484675
AUC: 0.6338885052392634


In [24]:
param_grid= {'n_estimators': [50, 100, 200]}

gs_ada_cc= GridSearchCV(estimator= ada_cc, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_ada_cc.fit(X_cc_train, y_cc_train)
print("Best Parameters:", gs_ada_cc.best_params_)
print("Best Precision Score:", gs_ada_cc.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'n_estimators': 200}
Best Precision Score: 0.9597634547029473


In [25]:
tuned_ada_cc= gs_ada_cc.best_estimator_
y_pred_tuned_ada_cc= tuned_ada_cc.predict(X_test)
y_pred_prob_tuned_ada_cc= tuned_ada_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_ada_cc, labels= tuned_ada_cc.classes_)
print(classification_report(y_test, y_pred_tuned_ada_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_ada_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_ada_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_ada_cc))

              precision    recall  f1-score   support

           0       0.95      0.03      0.05      8517
           1       0.10      0.99      0.18       912

    accuracy                           0.12      9429
   macro avg       0.53      0.51      0.12      9429
weighted avg       0.87      0.12      0.07      9429

Precision: 0.09807126511932004
Accuracy: 0.12090359529112313
AUC: 0.6290386847563901


--- GradientBoosting ---

In [26]:
from sklearn.ensemble import GradientBoostingClassifier
grb_cc= GradientBoostingClassifier(random_state=15)
grb_cc.fit(X_cc_train, y_cc_train)
y_pred_grb_cc= grb_cc.predict(X_test)
y_pred_prob_grb_cc= grb_cc.predict_proba(X_test)[:,1]


cm= confusion_matrix(y_test, y_pred_grb_cc, labels= grb_cc.classes_)
print(classification_report(y_test, y_pred_grb_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_grb_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_grb_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_grb_cc))

              precision    recall  f1-score   support

           0       0.98      0.04      0.08      8517
           1       0.10      0.99      0.18       912

    accuracy                           0.13      9429
   macro avg       0.54      0.52      0.13      9429
weighted avg       0.89      0.13      0.09      9429

Precision: 0.09974622089815734
Accuracy: 0.13384240110298018
AUC: 0.6560500001029932


In [27]:
param_grid= {'learning_rate': [0.01, 0.1, 0.2]}

gs_grb_cc= GridSearchCV(estimator= grb_cc, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_grb_cc.fit(X_cc_train, y_cc_train)

print("Best Parameters:", gs_grb_cc.best_params_)
print("Best Precision Score:", gs_grb_cc.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.2}
Best Precision Score: 0.9557133295551026


In [28]:
tuned_grb_cc= gs_grb_cc.best_estimator_
y_pred_tuned_grb_cc= tuned_grb_cc.predict(X_test)
y_pred_prob_tuned_grb_cc= tuned_grb_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_grb_cc, labels= tuned_grb_cc.classes_)
print(classification_report(y_test, y_pred_tuned_grb_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_grb_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_grb_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_grb_cc))

              precision    recall  f1-score   support

           0       0.95      0.04      0.08      8517
           1       0.10      0.98      0.18       912

    accuracy                           0.13      9429
   macro avg       0.53      0.51      0.13      9429
weighted avg       0.87      0.13      0.09      9429

Precision: 0.09880768381541179
Accuracy: 0.13246367589351998
AUC: 0.6417967728114462


--- XGB ---

In [29]:
from xgboost import XGBClassifier
xgb_cc= XGBClassifier(random_state=15)
xgb_cc.fit(X_cc_train, y_cc_train)
y_pred_xgb_cc= xgb_cc.predict(X_test)
y_pred_prob_xgb_cc= xgb_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_xgb_cc, labels= xgb_cc.classes_)
print(classification_report(y_test, y_pred_xgb_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_xgb_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_xgb_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_xgb_cc))

              precision    recall  f1-score   support

           0       0.95      0.04      0.09      8517
           1       0.10      0.98      0.18       912

    accuracy                           0.13      9429
   macro avg       0.53      0.51      0.13      9429
weighted avg       0.87      0.13      0.09      9429

Precision: 0.09888162994131325
Accuracy: 0.13490295895641108
AUC: 0.6056212523353706


In [30]:
params_XGBoost= {'learning_rate': [0.01, 0.1, 1.0]}

gs_xgb_cc= GridSearchCV(estimator= xgb_cc, param_grid= params_XGBoost, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_xgb_cc.fit(X_cc_train, y_cc_train)

print("Best Parameters:", gs_xgb_cc.best_params_)
print("Best Precision Score:", gs_xgb_cc.best_score_)

tuned_xgb_cc= gs_xgb_cc.best_estimator_
y_pred_tuned_xgb_cc= tuned_xgb_cc.predict(X_test)
y_pred_prob_tuned_xgb_cc= tuned_xgb_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_xgb_cc, labels= tuned_xgb_cc.classes_)
print(classification_report(y_test, y_pred_tuned_xgb_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_xgb_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_xgb_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_xgb_cc))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 1.0}
Best Precision Score: 0.9592192641282177
              precision    recall  f1-score   support

           0       0.95      0.05      0.09      8517
           1       0.10      0.98      0.18       912

    accuracy                           0.14      9429
   macro avg       0.53      0.51      0.14      9429
weighted avg       0.87      0.14      0.10      9429

Precision: 0.09911012235817575
Accuracy: 0.13882702301410543
AUC: 0.6010833724707448


--- LGBM ---

In [31]:
from lightgbm import LGBMClassifier
lgm_cc= LGBMClassifier(random_state=15)
lgm_cc.fit(X_cc_train, y_cc_train)
y_pred_lgm_cc= lgm_cc.predict(X_test)
y_pred_prob_lgm_cc= lgm_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_lgm_cc, labels= lgm_cc.classes_)
print(classification_report(y_test, y_pred_lgm_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lgm_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lgm_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lgm_cc))

[LightGBM] [Info] Number of positive: 3649, number of negative: 3649
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.169105 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2599
[LightGBM] [Info] Number of data points in the train set: 7298, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
              precision    recall  f1-score   support

           0       0.96      0.04      0.08      8517
           1       0.10      0.98      0.18       912

    accuracy                           0.13      9429
   macro avg       0.53      0.51      0.13      9429
weighted avg       0.88      0.13      0.09      9429

Precision: 0.0991060589338925
Accuracy: 0.13278184324954925
AUC: 0.5782770758792014


In [32]:
params_LGB= {'learning_rate': [0.001, 0.01, 0.1, 1.0],
             'num_leaves': [31, 127],
             'reg_alpha': [0.1, 0.5],
             'min_data_in_leaf': [30, 50, 100, 300, 400]}

gs_lgm_cc= GridSearchCV(estimator= lgm_cc, param_grid= params_LGB, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_lgm_cc.fit(X_cc_train, y_cc_train)

print("Best Parameters:", gs_lgm_cc.best_params_)
print("Best Precision Score:", gs_lgm_cc.best_score_)

tuned_lgm_cc= gs_lgm_cc.best_estimator_
y_pred_tuned_lgm_cc= tuned_lgm_cc.predict(X_test)
y_pred_prob_tuned_lgm_cc= tuned_lgm_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_lgm_cc, labels= tuned_lgm_cc.classes_)
print(classification_report(y_test, y_pred_tuned_lgm_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_lgm_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_lgm_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_lgm_cc))

Fitting 5 folds for each of 80 candidates, totalling 400 fits
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Info] Number of positive: 3649, number of negative: 3649
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005002 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2599
[LightGBM] [Info] Number of data points in the train set: 7298, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits w

Precision: 0.09962112770225094
Accuracy: 0.1411602502916534
AUC: 0.6109518578941189


--- CatBoost ---

In [33]:
from catboost import CatBoostClassifier
cat_cc= CatBoostClassifier(random_state=15)
cat_cc.fit(X_cc_train, y_cc_train)
y_pred_cat_cc= cat_cc.predict(X_test)
y_pred_prob_cat_cc= cat_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_cat_cc, labels= cat_cc.classes_)
print(classification_report(y_test, y_pred_cat_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_cat_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_cat_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_cat_cc))

Learning rate set to 0.024073
0:	learn: 0.6586823	total: 142ms	remaining: 2m 21s
1:	learn: 0.6182913	total: 153ms	remaining: 1m 16s
2:	learn: 0.5833375	total: 160ms	remaining: 53.2s
3:	learn: 0.5520819	total: 169ms	remaining: 42.1s
4:	learn: 0.5209602	total: 176ms	remaining: 35.1s
5:	learn: 0.4954481	total: 186ms	remaining: 30.8s
6:	learn: 0.4741558	total: 194ms	remaining: 27.5s
7:	learn: 0.4498805	total: 202ms	remaining: 25s
8:	learn: 0.4306255	total: 212ms	remaining: 23.3s
9:	learn: 0.4101218	total: 219ms	remaining: 21.6s
10:	learn: 0.3918949	total: 228ms	remaining: 20.5s
11:	learn: 0.3756204	total: 236ms	remaining: 19.5s
12:	learn: 0.3609736	total: 245ms	remaining: 18.6s
13:	learn: 0.3468516	total: 253ms	remaining: 17.8s
14:	learn: 0.3344862	total: 262ms	remaining: 17.2s
15:	learn: 0.3223231	total: 269ms	remaining: 16.6s
16:	learn: 0.3125299	total: 279ms	remaining: 16.1s
17:	learn: 0.3014751	total: 293ms	remaining: 16s
18:	learn: 0.2916783	total: 304ms	remaining: 15.7s
19:	learn: 0.

174:	learn: 0.0778866	total: 1.66s	remaining: 7.81s
175:	learn: 0.0776194	total: 1.67s	remaining: 7.8s
176:	learn: 0.0774597	total: 1.68s	remaining: 7.79s
177:	learn: 0.0773009	total: 1.68s	remaining: 7.78s
178:	learn: 0.0771073	total: 1.69s	remaining: 7.77s
179:	learn: 0.0769449	total: 1.7s	remaining: 7.75s
180:	learn: 0.0767952	total: 1.71s	remaining: 7.74s
181:	learn: 0.0766160	total: 1.72s	remaining: 7.73s
182:	learn: 0.0763837	total: 1.73s	remaining: 7.73s
183:	learn: 0.0761224	total: 1.74s	remaining: 7.72s
184:	learn: 0.0759876	total: 1.75s	remaining: 7.71s
185:	learn: 0.0758379	total: 1.76s	remaining: 7.7s
186:	learn: 0.0754073	total: 1.77s	remaining: 7.69s
187:	learn: 0.0752641	total: 1.78s	remaining: 7.71s
188:	learn: 0.0750473	total: 1.79s	remaining: 7.7s
189:	learn: 0.0748078	total: 1.8s	remaining: 7.69s
190:	learn: 0.0746245	total: 1.81s	remaining: 7.67s
191:	learn: 0.0743307	total: 1.82s	remaining: 7.66s
192:	learn: 0.0740889	total: 1.83s	remaining: 7.65s
193:	learn: 0.073

347:	learn: 0.0524697	total: 3.16s	remaining: 5.91s
348:	learn: 0.0523773	total: 3.16s	remaining: 5.9s
349:	learn: 0.0522993	total: 3.17s	remaining: 5.89s
350:	learn: 0.0521494	total: 3.18s	remaining: 5.88s
351:	learn: 0.0520616	total: 3.19s	remaining: 5.87s
352:	learn: 0.0519823	total: 3.19s	remaining: 5.86s
353:	learn: 0.0518875	total: 3.2s	remaining: 5.84s
354:	learn: 0.0518042	total: 3.21s	remaining: 5.83s
355:	learn: 0.0516792	total: 3.21s	remaining: 5.82s
356:	learn: 0.0515799	total: 3.22s	remaining: 5.8s
357:	learn: 0.0515133	total: 3.23s	remaining: 5.79s
358:	learn: 0.0514114	total: 3.24s	remaining: 5.78s
359:	learn: 0.0513050	total: 3.25s	remaining: 5.77s
360:	learn: 0.0511891	total: 3.25s	remaining: 5.76s
361:	learn: 0.0511073	total: 3.26s	remaining: 5.75s
362:	learn: 0.0510055	total: 3.28s	remaining: 5.75s
363:	learn: 0.0508649	total: 3.29s	remaining: 5.76s
364:	learn: 0.0507604	total: 3.31s	remaining: 5.76s
365:	learn: 0.0506853	total: 3.32s	remaining: 5.76s
366:	learn: 0.0

516:	learn: 0.0380002	total: 4.66s	remaining: 4.35s
517:	learn: 0.0379572	total: 4.67s	remaining: 4.34s
518:	learn: 0.0377910	total: 4.68s	remaining: 4.33s
519:	learn: 0.0377486	total: 4.68s	remaining: 4.33s
520:	learn: 0.0377096	total: 4.69s	remaining: 4.32s
521:	learn: 0.0376509	total: 4.7s	remaining: 4.31s
522:	learn: 0.0376079	total: 4.71s	remaining: 4.3s
523:	learn: 0.0375400	total: 4.72s	remaining: 4.29s
524:	learn: 0.0374996	total: 4.73s	remaining: 4.28s
525:	learn: 0.0374223	total: 4.74s	remaining: 4.27s
526:	learn: 0.0373577	total: 4.74s	remaining: 4.26s
527:	learn: 0.0372624	total: 4.75s	remaining: 4.25s
528:	learn: 0.0372361	total: 4.76s	remaining: 4.24s
529:	learn: 0.0372037	total: 4.77s	remaining: 4.23s
530:	learn: 0.0371619	total: 4.78s	remaining: 4.22s
531:	learn: 0.0370978	total: 4.79s	remaining: 4.21s
532:	learn: 0.0369595	total: 4.79s	remaining: 4.2s
533:	learn: 0.0369217	total: 4.8s	remaining: 4.19s
534:	learn: 0.0368712	total: 4.82s	remaining: 4.19s
535:	learn: 0.03

678:	learn: 0.0290075	total: 5.94s	remaining: 2.81s
679:	learn: 0.0289603	total: 5.95s	remaining: 2.8s
680:	learn: 0.0289057	total: 5.95s	remaining: 2.79s
681:	learn: 0.0288656	total: 5.96s	remaining: 2.78s
682:	learn: 0.0288153	total: 5.96s	remaining: 2.77s
683:	learn: 0.0287847	total: 5.97s	remaining: 2.76s
684:	learn: 0.0287404	total: 5.98s	remaining: 2.75s
685:	learn: 0.0286545	total: 5.99s	remaining: 2.74s
686:	learn: 0.0286074	total: 6s	remaining: 2.73s
687:	learn: 0.0285782	total: 6s	remaining: 2.72s
688:	learn: 0.0284960	total: 6.01s	remaining: 2.71s
689:	learn: 0.0284570	total: 6.01s	remaining: 2.7s
690:	learn: 0.0284200	total: 6.02s	remaining: 2.69s
691:	learn: 0.0283929	total: 6.03s	remaining: 2.68s
692:	learn: 0.0283582	total: 6.04s	remaining: 2.67s
693:	learn: 0.0283246	total: 6.04s	remaining: 2.66s
694:	learn: 0.0282804	total: 6.05s	remaining: 2.65s
695:	learn: 0.0282477	total: 6.06s	remaining: 2.65s
696:	learn: 0.0282055	total: 6.07s	remaining: 2.64s
697:	learn: 0.028159

845:	learn: 0.0226748	total: 7.23s	remaining: 1.32s
846:	learn: 0.0226390	total: 7.24s	remaining: 1.31s
847:	learn: 0.0226160	total: 7.25s	remaining: 1.3s
848:	learn: 0.0225968	total: 7.26s	remaining: 1.29s
849:	learn: 0.0225463	total: 7.27s	remaining: 1.28s
850:	learn: 0.0225075	total: 7.28s	remaining: 1.27s
851:	learn: 0.0224776	total: 7.29s	remaining: 1.26s
852:	learn: 0.0224300	total: 7.3s	remaining: 1.26s
853:	learn: 0.0224024	total: 7.3s	remaining: 1.25s
854:	learn: 0.0223655	total: 7.31s	remaining: 1.24s
855:	learn: 0.0223310	total: 7.32s	remaining: 1.23s
856:	learn: 0.0223076	total: 7.33s	remaining: 1.22s
857:	learn: 0.0222580	total: 7.34s	remaining: 1.21s
858:	learn: 0.0222244	total: 7.34s	remaining: 1.21s
859:	learn: 0.0222095	total: 7.36s	remaining: 1.2s
860:	learn: 0.0221770	total: 7.36s	remaining: 1.19s
861:	learn: 0.0221429	total: 7.37s	remaining: 1.18s
862:	learn: 0.0221038	total: 7.38s	remaining: 1.17s
863:	learn: 0.0220678	total: 7.39s	remaining: 1.16s
864:	learn: 0.02

In [34]:
params_CatBoost= {
                  'depth': [3,5,10],
                  'learning_rate' : [0.01,0.1,1],
                  'iterations' : [5,10,50,100]
                 }

gs_cat_cc= GridSearchCV(estimator= cat_cc, param_grid= params_CatBoost, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_cat_cc.fit(X_cc_train, y_cc_train)

print("Best Parameters:", gs_cat_cc.best_params_)
print("Best Precision Score:", gs_cat_cc.best_score_)

tuned_cat_cc= gs_cat_cc.best_estimator_
y_pred_tuned_cat_cc= tuned_cat_cc.predict(X_test)
y_pred_prob_tuned_cat_cc= tuned_cat_cc.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_cat_cc, labels= tuned_cat_cc.classes_)
print(classification_report(y_test, y_pred_tuned_cat_cc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_cat_cc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_cat_cc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_cat_cc))

Fitting 5 folds for each of 36 candidates, totalling 180 fits
0:	learn: 0.3308438	total: 3.98ms	remaining: 394ms
1:	learn: 0.2295447	total: 8.54ms	remaining: 418ms
2:	learn: 0.1959434	total: 11.9ms	remaining: 386ms
3:	learn: 0.1725915	total: 15.5ms	remaining: 373ms
4:	learn: 0.1504219	total: 19.9ms	remaining: 379ms
5:	learn: 0.1323418	total: 23.9ms	remaining: 375ms
6:	learn: 0.1214681	total: 27.6ms	remaining: 366ms
7:	learn: 0.0993563	total: 31.3ms	remaining: 360ms
8:	learn: 0.0931171	total: 34.5ms	remaining: 349ms
9:	learn: 0.0886818	total: 37.5ms	remaining: 338ms
10:	learn: 0.0848216	total: 40.6ms	remaining: 328ms
11:	learn: 0.0819076	total: 43.6ms	remaining: 320ms
12:	learn: 0.0793490	total: 46.9ms	remaining: 314ms
13:	learn: 0.0757940	total: 50.5ms	remaining: 310ms
14:	learn: 0.0732943	total: 53.7ms	remaining: 304ms
15:	learn: 0.0685566	total: 57.3ms	remaining: 301ms
16:	learn: 0.0672367	total: 60.7ms	remaining: 296ms
17:	learn: 0.0648700	total: 66.5ms	remaining: 303ms
18:	learn: 0

--- Visualisation ---

In [35]:
print('Logistic Regression Precision:', precision_score(y_test, y_pred_lr_cc, zero_division= 0))
print('Decision Tree Precision:', precision_score(y_test, y_pred_dt_cc, zero_division= 0))
print('Tuned Decision Tree Precision:', precision_score(y_test, y_pred_tuned_dt_cc, zero_division= 0))
print('KNeighborsClassifier Precision:', precision_score(y_test, y_pred_knn_cc, zero_division= 0))
print('Tuned KNeighborsClassifier Precision:', precision_score(y_test, y_pred_tuned_knn_cc, zero_division= 0))
print('GaussianNB Precision:', precision_score(y_test, y_pred_nb_cc, zero_division= 0))
print('SVM Precision:', precision_score(y_test, y_pred_svc_cc, zero_division= 0))
print('Random Forest Precision:', precision_score(y_test, y_pred_rf_cc, zero_division= 0))
print('Tuned Random Forest Precision:', precision_score(y_test, y_pred_tuned_rf_cc, zero_division= 0))
print('AdaBoost Precision:', precision_score(y_test, y_pred_ada_cc, zero_division= 0))
print('Tuned AdaBoost Precision:', precision_score(y_test, y_pred_tuned_ada_cc, zero_division= 0))
print('GradientBoosting Precision:', precision_score(y_test, y_pred_grb_cc, zero_division= 0))
print('Tuned GradientBoosting Precision:', precision_score(y_test, y_pred_tuned_grb_cc, zero_division= 0))
print('XGB Precision:', precision_score(y_test, y_pred_xgb_cc, zero_division= 0))
print('Tuned XGB Precision:', precision_score(y_test, y_pred_tuned_xgb_cc, zero_division= 0))
print('LGBM Precision:', precision_score(y_test, y_pred_lgm_cc, zero_division= 0))
print('Tuned LGBM Precision:', precision_score(y_test, y_pred_tuned_lgm_cc, zero_division= 0))
print('CatBoost Precision:', precision_score(y_test, y_pred_cat_cc, zero_division= 0))
print('Tuned CatBoost Precision:', precision_score(y_test, y_pred_tuned_cat_cc, zero_division= 0))

Logistic Regression Precision: 0.23249211356466876
Decision Tree Precision: 0.10629629629629629
Tuned Decision Tree Precision: 0.10783329148882752
KNeighborsClassifier Precision: 0.22462562396006655
Tuned KNeighborsClassifier Precision: 0.1841917873249466
GaussianNB Precision: 0.13863031914893617
SVM Precision: 0.13152804642166344
Random Forest Precision: 0.10183527305282006
Tuned Random Forest Precision: 0.10191296565611366
AdaBoost Precision: 0.09788446782856516
Tuned AdaBoost Precision: 0.09807126511932004
GradientBoosting Precision: 0.09974622089815734
Tuned GradientBoosting Precision: 0.09880768381541179
XGB Precision: 0.09888162994131325
Tuned XGB Precision: 0.09911012235817575
LGBM Precision: 0.0991060589338925
Tuned LGBM Precision: 0.09962112770225094
CatBoost Precision: 0.09915076651593692
Tuned CatBoost Precision: 0.09934217861523023


In [36]:
precision_scores= {
                    'Logistic Regression Precision:': precision_score(y_test, y_pred_lr_cc, zero_division= 0),
                    'Decision Tree Precision:': precision_score(y_test, y_pred_dt_cc, zero_division= 0),
                    'Tuned Decision Tree Precision:': precision_score(y_test, y_pred_tuned_dt_cc, zero_division= 0),
                    'KNeighborsClassifier Precision:': precision_score(y_test, y_pred_knn_cc, zero_division= 0),
                    'Tuned KNeighborsClassifier Precision:': precision_score(y_test, y_pred_tuned_knn_cc, zero_division= 0),
                    'GaussianNB Precision:': precision_score(y_test, y_pred_nb_cc, zero_division= 0),
                    'SVM Precision:': precision_score(y_test, y_pred_svc_cc, zero_division= 0),
                    'Random Forest Precision:': precision_score(y_test, y_pred_rf_cc, zero_division= 0),
                    'Tuned Random Forest Precision:': precision_score(y_test, y_pred_tuned_rf_cc, zero_division= 0),
                    'AdaBoost Precision:': precision_score(y_test, y_pred_ada_cc, zero_division= 0),
                    'Tuned AdaBoost Precision:': precision_score(y_test, y_pred_tuned_ada_cc, zero_division= 0),
                    'GradientBoosting Precision:': precision_score(y_test, y_pred_grb_cc, zero_division= 0),
                    'Tuned GradientBoosting Precision:': precision_score(y_test, y_pred_tuned_grb_cc, zero_division= 0),
                    'XGB Precision:': precision_score(y_test, y_pred_xgb_cc, zero_division= 0),
                    'Tuned XGB Precision:': precision_score(y_test, y_pred_tuned_xgb_cc, zero_division= 0),
                    'LGBM Precision:': precision_score(y_test, y_pred_lgm_cc, zero_division= 0),
                    'Tuned LGBM Precision:': precision_score(y_test, y_pred_tuned_lgm_cc, zero_division= 0),
                    'CatBoost Precision:': precision_score(y_test, y_pred_cat_cc, zero_division= 0),
                    'Tuned CatBoost Precision:': precision_score(y_test, y_pred_tuned_cat_cc, zero_division= 0)
                  }

cc_precision= pd.DataFrame(list(precision_scores.items()), columns= ['Model', 'Precision Score'])
cc_precision= cc_precision.sort_values(by= 'Precision Score', ascending=False)
print(cc_precision)

                                    Model  Precision Score
0          Logistic Regression Precision:         0.232492
3         KNeighborsClassifier Precision:         0.224626
4   Tuned KNeighborsClassifier Precision:         0.184192
5                   GaussianNB Precision:         0.138630
6                          SVM Precision:         0.131528
2          Tuned Decision Tree Precision:         0.107833
1                Decision Tree Precision:         0.106296
8          Tuned Random Forest Precision:         0.101913
7                Random Forest Precision:         0.101835
11            GradientBoosting Precision:         0.099746
16                  Tuned LGBM Precision:         0.099621
18              Tuned CatBoost Precision:         0.099342
17                    CatBoost Precision:         0.099151
14                   Tuned XGB Precision:         0.099110
15                        LGBM Precision:         0.099106
13                         XGB Precision:         0.0988

In [37]:
accuracy_scores= {
                    'Logistic Regression Accuracy:': accuracy_score(y_test, y_pred_lr_cc),
                    'Decision Tree Accuracy:': accuracy_score(y_test, y_pred_dt_cc),
                    'Tuned Decision Tree Accuracy:': accuracy_score(y_test, y_pred_tuned_dt_cc),
                    'KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_knn_cc),
                    'Tuned KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_tuned_knn_cc),
                    'GaussianNB Accuracy:': accuracy_score(y_test, y_pred_nb_cc),
                    'SVM Accuracy:': accuracy_score(y_test, y_pred_svc_cc),
                    'Random Forest Accuracy:': accuracy_score(y_test, y_pred_rf_cc),
                    'Tuned Random Forest Accuracy:': accuracy_score(y_test, y_pred_tuned_rf_cc),
                    'AdaBoost Accuracy:': accuracy_score(y_test, y_pred_ada_cc),
                    'Tuned AdaBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_ada_cc),
                    'GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_grb_cc),
                    'Tuned GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_tuned_grb_cc),
                    'XGB Accuracy:': accuracy_score(y_test, y_pred_xgb_cc),
                    'Tuned XGB Accuracy:': accuracy_score(y_test, y_pred_tuned_xgb_cc),
                    'LGBM Accuracy:': accuracy_score(y_test, y_pred_lgm_cc),
                    'Tuned LGBM Accuracy:': accuracy_score(y_test, y_pred_tuned_lgm_cc),
                    'CatBoost Accuracy:': accuracy_score(y_test, y_pred_cat_cc),
                    'Tuned CatBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_cat_cc)
                  }

cc_accuracy= pd.DataFrame(list(accuracy_scores.items()), columns= ['Model', 'Accuracy Score'])
cc_accuracy= cc_accuracy.sort_values(by= 'Accuracy Score', ascending=False)
print(cc_accuracy)

                                   Model  Accuracy Score
6                          SVM Accuracy:        0.862870
3         KNeighborsClassifier Accuracy:        0.797964
0          Logistic Regression Accuracy:        0.723407
4   Tuned KNeighborsClassifier Accuracy:        0.621063
5                   GaussianNB Accuracy:        0.442147
2          Tuned Decision Tree Accuracy:        0.240641
1                Decision Tree Accuracy:        0.226853
7                Random Forest Accuracy:        0.148584
8          Tuned Random Forest Accuracy:        0.148478
16                  Tuned LGBM Accuracy:        0.141160
18              Tuned CatBoost Accuracy:        0.141054
14                   Tuned XGB Accuracy:        0.138827
13                         XGB Accuracy:        0.134903
11            GradientBoosting Accuracy:        0.133842
15                        LGBM Accuracy:        0.132782
12      Tuned GradientBoosting Accuracy:        0.132464
17                    CatBoost 

In [38]:
auc_scores= {
                    'Logistic Regression AUC:': roc_auc_score(y_test, y_pred_prob_lr_cc),
                    'Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_dt_cc),
                    'Tuned Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_tuned_dt_cc),
                    'KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_knn_cc),
                    'Tuned KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_tuned_knn_cc),
                    'GaussianNB AUC:': roc_auc_score(y_test, y_pred_prob_nb_cc),
                    'SVM AUC:': roc_auc_score(y_test, y_pred_prob_svc_cc),
                    'Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_rf_cc),
                    'Tuned Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_tuned_rf_cc),
                    'AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_ada_cc),
                    'Tuned AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_ada_cc),
                    'GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_grb_cc),
                    'Tuned GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_tuned_grb_cc),
                    'XGB AUC:': roc_auc_score(y_test, y_pred_prob_xgb_cc),
                    'Tuned XGB AUC:': roc_auc_score(y_test, y_pred_prob_tuned_xgb_cc),
                    'LGBM AUC:': roc_auc_score(y_test, y_pred_prob_lgm_cc),
                    'Tuned LGBM AUC:': roc_auc_score(y_test, y_pred_prob_tuned_lgm_cc),
                    'CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_cat_cc),
                    'Tuned CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_cat_cc)
                  }

cc_auc= pd.DataFrame(list(auc_scores.items()), columns= ['Model', 'AUC Score'])
cc_auc= cc_auc.sort_values(by= 'AUC Score', ascending=False)
print(cc_auc)

                              Model  AUC Score
0          Logistic Regression AUC:   0.831938
5                   GaussianNB AUC:   0.795081
4   Tuned KNeighborsClassifier AUC:   0.787734
3         KNeighborsClassifier AUC:   0.731715
11            GradientBoosting AUC:   0.656050
8          Tuned Random Forest AUC:   0.653873
7                Random Forest AUC:   0.649609
12      Tuned GradientBoosting AUC:   0.641797
17                    CatBoost AUC:   0.639475
9                     AdaBoost AUC:   0.633889
10              Tuned AdaBoost AUC:   0.629039
16                  Tuned LGBM AUC:   0.610952
6                          SVM AUC:   0.605883
13                         XGB AUC:   0.605621
14                   Tuned XGB AUC:   0.601083
2          Tuned Decision Tree AUC:   0.583559
18              Tuned CatBoost AUC:   0.580452
15                        LGBM AUC:   0.578277
1                Decision Tree AUC:   0.547066
